<a href="https://colab.research.google.com/github/eisbetterthanpi/python/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

training_data = datasets.FashionMNIST(root="data", train=True, download=True,transform=ToTensor(),)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor(),)
# print(training_data)

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# test_dataloader: #image, label


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 1, 3, padding=1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            # nn.Conv2d(1, 1, 3, padding=1),
            # nn.Dropout2d(),
            # nn.MaxPool2d(2, 2),
            # nn.ReLU(),
            # # nn.Conv2d(1, 1, 3, padding=1, stride=2), nn.ReLU(),
            # nn.Flatten(),
            # nn.Linear(49, 10), nn.ReLU(),
            # nn.Linear(28, 10),
        )
        self.linear_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(49, 10),
            # nn.Dropout(),
            # nn.LogSoftmax(),
            # # nn.BatchNorm2d(4),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x=x.view(-1,49)
        x = self.linear_stack(x)
        return x


# softmax = nn.Softmax(dim=1)
# pred_probab = softmax(logits)
model = NeuralNetwork().to(device) # create an instance and move it to device (cache?)
print(model)

Using cpu device
NeuralNetwork(
  (conv_stack): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (linear_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=49, out_features=10, bias=True)
  )
)


In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X = X.unsqueeze(1) #? https://stackoverflow.com/questions/62427615/runtimeerror-expected-4-dimensional-input-for-4-dimensional-weight-x-but-got-3
        X, y = X.to(device), y.to(device)
        pred = model(X) # Compute prediction error
        loss = loss_fn(pred, y)
        optimizer.zero_grad() # Backpropagation
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # X = X.unsqueeze(1) #?
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [ ]:

epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))


Epoch 1
-------------------------------


ValueError: ignored

In [ ]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot",]

model.eval()

import random
n=random.randint(0,1000)
print(n)
x, y = test_data[n][0], test_data[n][1]
x = x.unsqueeze(1) #?

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


